# Streaming data with Pysep to be used in MTUQ
**Félix Rodríguez-Cardozo and Jochen Braunmiller**

The following Notebook provides an example for retrieving data from the [FDSN web services](https://www.fdsn.org/webservices/) via [Pysep](https://github.com/adjtomo/pysep) and using it for estimating the seismic moment tensor in [MTUQ](https://github.com/uafgeotools/mtuq).

Before running this example, you must have installed [Pysep](https://pysep.readthedocs.io/en/latest/) and [MTUQ](https://uafgeotools.github.io/mtuq/install/index.html) and this notebook had to have been opened **after activating the pysep environment** 

### 1. Check the examples available in Pysep
Pysep includes examples for retrieving and pre-processing data. To see those examples, run the next cell. 

In [ ]:
import os
main_dir = os.getcwd()
! pysep -l

Those examples have configuration files (*.yaml) ready for streaming and processing data. You can see their content in [Pysep configuration file examples](https://github.com/adjtomo/pysep/tree/master/pysep/configs/mtuq_workshop_2022) 

### 2. Retrieving data for North Korean nuclear test waveforms

Pysep can retrieve data ready to use in MTUQ. To do so, the following lines should be added to the configuration file:

**_legacy_naming: true**

**write_files: inv,event,stream,sac,weights_code**

Normally, you can stream and pre-process data from any of the shown examples, such as the North Korean one, by typing in the terminal:

**pysep -p mtuq_workshop_2022 -e 2017-09-03T033001_NORTH_KOREA.yaml**

However, for streaming and pre-processing the data for running MTUQ, we need to modify yaml files and add the aforementioned lines. The modified configuration file is given with this example. By executing the following cell, Pysep will read the local configuration file and stream and pre-process data for MTUQ. 

In [ ]:
! pysep -c 2017-09-03T033001_NORTH_KOREA.yaml

### 3. Check the retrieved data
If the data was retrieved successfully in the same directory where  this Notebook resides, you must find the directory **20170903033001760**. 

By running the next cell, you can explore the directory content:

In [ ]:
! ls 20170903033001760

To run MTUQ, the most relevant files are the **SAC** files (such as 20170903033001760.IC.MDJ.00.BH.r) and the **weights.dat**. However, the record_section.png and the station_map.png are very useful for seeing the azimuthal coverage and the quality of the retrieved seismograms. 

Opening the **record_section** file will provide a quick insight into what data should be included in the moment tensor estimation. Use the next cell to see the event record section. 

In [ ]:
from IPython.display import Image
Image("20170903033001760/record_section.png")

### 4. Running MTUQ with the retrieved data

The most important steps for running MTUQ were already  performed by Pysep: (1) Remove the instrumental response, (2) Complete SAC headers with earthquake and station locations,(3) rotate traces to transverse and radial and (4) write the weights.dat file. The latter is the input file that indicates what stations, components, and parts of the seismogram (body, surface waves) will be included in the MTUQ moment tensor estimation. 

The next step is to run MTUQ. Therefore, you need to change the environment and activate the MTUQ one. This cannot be done in this notebook. Therefore, open a terminal window, go to the directory where this notebook is located, and activate MTUQ (type in the terminal **conda activate mtuq**). If you need it run the next cell to retrieve the location of this notebook in your computer:

In [ ]:
import os
print(os.getcwd())

#### 4.1 Scripts for running MTUQ

This notebook comes with two scripts for running MTUQ:

- **GridSearch.DoubleCouple_SW_BW_options.py**: for a double-couple grid-search using body and surface waves.
- **GridSearch.FMT_SW_BW_options.py**: for a full moment tensor grid-search using body and surface waves.

These scripts stream pre-calculated the Green Functions based on the [ak135](https://ds.iris.edu/ds/products/emc-ak135-f/) velocity model. In addition, the scripts were designed to minimize the modification of the code and hence, the basic seismic source and some of the grid-search parameters can be parsed. 

#### 4.2 Double Couple Grid-Search
In the terminal where you activated MTUQ, type:

***python GridSearch.DoubleCouple_SW_BW_options.py -h***

You will see the following menu explaining how to parse the parameters for tunning the moment tensor estimation:

Input event info run MTUQ

optional arguments:
  -h, --help    show this help message and exit
  
  -event EVENT  Event ID (event directory must be in main dir): -event 20140823183304000 
  
  -evla EVLA    Latitude: -evla 64.68 
  
  -evlo EVLO    longitude: -evla -98.2 
  
  -evdp EVDP    Depth in m: -evdp 1000.0
  
  -mw MW        Magnitude: -mw 4.9
  
  -time TIME    Origin time: -time 2014-08-25T16:19:03.00000Z
  
  -np NP        Number of points per axis: -np 10
  
  -fb FB        Frequency band for filtering data in seconds (body_waves/surface_waves): -fb 3-15/15-33
  
  -wl WL        Window length in seconds (body_waves/surface_waves): -wl 30/200
  
  -ts TS        Time shift limits in seconds (body_waves/surface_waves): -ts 5/15 (means a +/-5s and +/-15s time shift)

For the event in this example, a reasonable setup of the MTUQ grid-search could be:

***mpirun -np 4 python GridSearch.DoubleCouple_SW_BW_options.py -event 20170903033001760 -evla 41.332401275634766 -evlo 129.02969360351562 -evdp 1000.0 -mw 5.3 -time 2017-09-03T03:30:01.000000Z -np 50 -fb 3-10/30-70 -wl 5/300 -ts 5/15***

the **mpirun -np 4** command shows how to run the script in parallel using 4 processes, which could be useful for comprehensive grid-searches (many points per axis). Be sure that your computer supports at least 4 processes. Otherwise, reduce the number of processes or simply omit the mpirun command. 

If your MTUQ run is worked, execute the next cells for seing the waveform fits and the misfit distribution

In [ ]:
Image("20170903033001760DC_waveforms.png")

In [ ]:
Image("20170903033001760DC_misfit.png")

#### 4.3 Full Moment Tensor Grid-Search

The **GridSearch.FMT_SW_BW_options.py** works in the same way as the double couple grid search script. However, for a reasonable time run, the number of point-per-axis should be reduced since the number of freedom degrees increases in this case. 

The following configuration of the MTUQ script should produce a fairy well results:

***mpirun -np 4 python GridSearch.FMT_SW_BW_options.py -event 20170903033001760 -evla 41.332401275634766 -evlo 129.02969360351562 -evdp 1000.0 -mw 5.3 -time 2017-09-03T03:30:01.000000Z -np 11 -fb 3-10/30-70 -wl 5/300 -ts 5/15***

After running MTUQ, run the next cells to see thE waveform fit and the misfit distribution in the lune-plot. 

In [ ]:
Image("20170903033001760FMT_waveforms.png")

In [ ]:
Image("20170903033001760FMT_misfit_mt.png")

Run the next cell if you want to start over

In [ ]:
! rm 20170903033001760DC_*
! rm 20170903033001760FMT_*